In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
import argparse
import json
import os
from collections import OrderedDict
import torch
import csv
import util
from transformers import DistilBertTokenizerFast, AutoTokenizer
from transformers import DistilBertForQuestionAnswering, AutoModelForQuestionAnswering
from transformers import AdamW
from tensorboardX import SummaryWriter

from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from args import get_train_test_args
from train import prepare_eval_data, prepare_train_data, read_and_process, get_dataset
from util import compute_f1, compute_em
from mymodel.model import MyModel

from tqdm import tqdm

C:\Users\user\AppData\Local\Temp\ipykernel_1776\3377161591.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### 함수 정의

In [2]:
def read_and_process(tokenizer, dataset_dict, dir_name, dataset_name, split):
    #TODO: cache this if possible
    cache_path = f'{dir_name}/{dataset_name}_encodings.pt'
    if os.path.exists(cache_path) and not True:
        tokenized_examples = util.load_pickle(cache_path)
    else:
        if split=='train':
            tokenized_examples = prepare_train_data(dataset_dict, tokenizer)
        else:
            tokenized_examples = prepare_eval_data(dataset_dict, tokenizer)
        util.save_pickle(tokenized_examples, cache_path)
    return tokenized_examples

def get_dataset(datasets, data_dir, tokenizer, split_name):
    datasets = datasets.split(',')
    dataset_dict = None
    dataset_name=''
    for dataset in datasets:
        dataset_name += f'_{dataset}'
        dataset_dict_curr = util.read_squad(f'{data_dir}/{dataset}')
        dataset_dict = util.merge(dataset_dict, dataset_dict_curr)
    data_encodings = read_and_process(tokenizer, dataset_dict, data_dir, dataset_name, split_name)
    return util.QADataset(data_encodings, train=(split_name=='train')), dataset_dict

def evaluate(model, data_loader, data_dict, return_preds=False, split='validation'):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    model.eval()
    pred_dict = {}
    all_start_logits = []
    all_end_logits = []
    with torch.no_grad(), \
            tqdm(total=len(data_loader.dataset)) as progress_bar:
        for batch in data_loader:
            # Setup for forward
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            batch_size = len(input_ids)
            outputs = model(input_ids, attention_mask=attention_mask)
            # Forward
            start_logits, end_logits = outputs.start_logits, outputs.end_logits
            # TODO: compute loss

            all_start_logits.append(start_logits)
            all_end_logits.append(end_logits)
            progress_bar.update(batch_size)

    # Get F1 and EM scores
    start_logits = torch.cat(all_start_logits).cpu().numpy()
    end_logits = torch.cat(all_end_logits).cpu().numpy()
    preds = util.postprocess_qa_predictions(data_dict,
                                             data_loader.dataset.encodings,
                                             (start_logits, end_logits))
    preds = util.postprocess_qa_predictions(data_dict,
                                                 data_loader.dataset.encodings,
                                                 (start_logits, end_logits))
    if split == 'validation':
        results = util.eval_dicts(data_dict, preds)
        results_list = [('F1', results['F1']),
                        ('EM', results['EM'])]
    else:
        results_list = [('F1', -1.0),
                        ('EM', -1.0)]
    results = OrderedDict(results_list)
    if return_preds:
        return preds, results
    return results

In [3]:
# eval_dir = 'datasets/indomain_val'
# eval_datasets = 'squad,nat_questions,newsqa'

eval_dir = 'datasets/oodomain_val'
eval_datasets = 'race,relation_extraction,duorc'

batch_size = 16

In [4]:
tokenizers = {'TinyBERT':"deepset/tinybert-6l-768d-squad2", 'DistilBERT':'distilbert-base-uncased', 'BERT':'bert-base-uncased', 'SqueezeBERT':'squeezebert/squeezebert-uncased'}

path = './save/00.aa/05.Ablation-ab/'

ds = os.listdir(path)

f1s = []
ems = []
names = []

for d in ds:

    tokenizer = AutoTokenizer.from_pretrained('deepset/tinybert-6l-768d-squad2')

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    split_name = 'test' if 'test' in eval_dir else 'validation'

    checkpoint_path = os.path.join(path+'/{}'.format(d), 'checkpoint')
    
    print(checkpoint_path)

    model = AutoModelForQuestionAnswering.from_pretrained(checkpoint_path)

    model.to(device)
    
    eval_dataset, eval_dict = get_dataset(eval_datasets, eval_dir, tokenizer, split_name)

    eval_loader = DataLoader(eval_dataset,batch_size=batch_size,sampler=SequentialSampler(eval_dataset))
    
    eval_preds, eval_scores = evaluate(model, eval_loader,eval_dict, return_preds=True, split=split_name)
    
    f1 = eval_scores['F1']
    em = eval_scores['EM']
        
    f1s.append(f1)
    ems.append(em)
    names.append(d)

./save/00.aa/05.Ablation-ab//0.0and0.0-01\checkpoint


100%|██████████████████████████████████████████████████████████████████████████████| 382/382 [00:00<00:00, 2633.87it/s]


./save/00.aa/05.Ablation-ab//0.0and0.1-01\checkpoint


100%|██████████████████████████████████████████████████████████████████████████████| 382/382 [00:00<00:00, 2652.18it/s]


./save/00.aa/05.Ablation-ab//0.0and0.2-01\checkpoint


...

./save/00.aa/05.Ablation-ab//0.9and0.0-01\checkpoint


100%|██████████████████████████████████████████████████████████████████████████████| 382/382 [00:00<00:00, 2417.19it/s]


./save/00.aa/05.Ablation-ab//1.0and0.0-01\checkpoint


100%|██████████████████████████████████████████████████████████████████████████████| 382/382 [00:00<00:00, 2479.95it/s]


ValueError: could not convert string to float: 'log_train.txt'

In [5]:
cnt = 0
for n, f, e in zip(names, f1s, ems):
    
    print(cnt, n, f, e)
    
    cnt += 1

0 clsent 50.1692542778479 35.602094240837694
1 clssim 49.48590136209478 33.246073298429316
2 onlycls 49.607004993576425 33.769633507853406
